In [94]:
from bs4 import BeautifulSoup as bsoup, Tag
import requests
import datetime
import time
import webbrowser as browser
import pandas as pd
import numpy as np
chrome_path = "open -a /Applications/Google\ Chrome.app %s"

In [95]:
indeed_mle_csv = "indeed/indeed_mle.csv"
df = pd.read_csv(indeed_mle_csv)
df.head()

,Position,Description,Url


In [96]:
def saveWebpage(prefix,response):
    """Save webpage locally as HTML and returns file location as string"""
    now = datetime.datetime.now().strftime("%Y-%m-%d_%H.%M")
    doc_name = "indeed/{prefix}_{time}.html".format(prefix=prefix,time=now)
    #https://stackoverflow.com/questions/31126596/saving-response-from-requests-to-file
    with open(doc_name, mode="wb+") as localfile:
        localfile.write(response.content)
    return doc_name

def getUrlsFromPage(website,params,prefix):
    response = requests.request("GET", website, params=params)
    page_location = saveWebpage(prefix,response)
    soup = bsoup(open(page_location), "html.parser")
    raw_urls = soup.find_all("a",class_="turnstileLink")
#     print(len(soup.find("div",class_="pagination").findChildren("a",recursive=False)))
    more_urls = True if soup.find("div",class_="pagination").findChildren("a",recursive=False,limit=1) else False
    return raw_urls, more_urls

def makeUrlsVisitable(urls):
    visitable_urls = []
    #other endpoints I found are /pagead and /cmp
    acceptable_endpoints = ["rc"]
    #https://stackoverflow.com/questions/36300158/python-split-text-after-the-second-occurrence-of-character
    for url in urls:
        if (type(url) is Tag and (url["href"].split("/", 2))[1] in acceptable_endpoints):
            visitable_urls.append("https://www.indeed.com"+url["href"])
    return visitable_urls
    

def getAllUrls(website,params,prefix):
    page_num = 0
    all_urls = set()
    page_urls, more_urls = getUrlsFromPage(website,params,prefix)
    if page_urls:
        print("page {page_num}: {urls_on_page} urls on page".format(page_num=1,urls_on_page=len(page_urls)))
        u = makeUrlsVisitable(page_urls)
        for url in u:
            all_urls.add(url)
        while more_urls and page_num < 5: #self rate limiting
            params["start"] = str(int(params["start"])+50) #get next 50 results
            page_urls, more_urls = getUrlsFromPage(website,params,prefix)
            page_num += 1
#             print("page {page_num}: {urls_on_page} urls on page".format(page_num=page_num,urls_on_page=len(page_urls)))
            u = makeUrlsVisitable(page_urls)
            for url in u:
                all_urls.add(url)
            time.sleep(1)
    return all_urls
    
    
def getJobsData(urls):
    """Gets job data from each url and returns Numpy matrix of data"""
    data = np.array([])
    return data
    
def saveJobsData(data):
    """Saves jobs data to Pandas dataframe and from there to CSV file"""
    return None


In [97]:
#get MLE data
website = "https://www.indeed.com/jobs"
# mle_params = {"q":"machine+learning+engineer","l":"San+Francisco+Bay+Area,+CA","explvl":"entry_level"}
# mle_params = {"as_and":"","as_phr":"Machine+Learning+Engineer","as_any":"","as_not":"","as_ttl":"",
#               "as_cmp":"","jt":"fulltime","st":"","as_src":"","salary":"","radius":"50",
#               "l":"San+Francisco+Bay+Area%2C+CA","fromage":"any","limit":"50","sort":"","psf":"advsrch","start":"0"}

mle_params = {"as_and":"","as_phr":"Machine+Learning+Engineer","as_any":"","as_not":"",
 "as_ttl":"","as_cmp":"","jt":"fulltime","st":"","as_src":"","salary":"","radius":"50","l":"San Francisco Bay Area, CA",
 "fromage":"any","limit":"50","sort":"","psf":"advsrch","start":"0"}

# {"as_and":"Machine+Learning+Engineer","as_phr":"Machine+Learning+Engineer","as_any":"","as_not":"",
#               "as_ttl":"","as_cmp":"","jt":"fulltime","st":"","as_src":"","salary":"","radius":"50",
#               "l":"San+Francisco+Bay+Area%2C+CA","fromage":"any","limit":"50","sort":"","psf":"advsrch"}
mle_prefix = "mle" 

In [98]:
urls = getAllUrls(website,mle_params,mle_prefix)
#https://stackoverflow.com/questions/8466014/how-to-convert-a-python-set-to-a-numpy-array
urls = list(urls)
num_urls = len(urls)
print(num_urls)
for url in urls:
    df.loc[len(df)] = [None,None,url.strip()]

page 1: 58 urls on page
156


In [99]:
df.to_csv(indeed_mle_csv, encoding='utf-8', index=False)

In [100]:
for i in range(10):
    url = df.iloc[i]["Url"]
    print(url)
    browser.get(chrome_path).open_new_tab(url)

https://www.indeed.com/rc/clk?jk=ae20548c6763d6e4&fccid=ae6e171391e978d5&vjs=3
https://www.indeed.com/rc/clk?jk=2f995bb9c1a1df6b&fccid=6e9166ecf7604472&vjs=3
https://www.indeed.com/rc/clk?jk=09e77269e281ba5e&fccid=849cecf89c5fe512&vjs=3
https://www.indeed.com/rc/clk?jk=1848b64b8db92063&fccid=3433aab0a6573d98&vjs=3
https://www.indeed.com/rc/clk?jk=47f8abbfe2603d8e&fccid=7d28c1874e2d22be&vjs=3
https://www.indeed.com/rc/clk?jk=c0930d01caa5d3be&fccid=c8100afc42779772&vjs=3
https://www.indeed.com/rc/clk?jk=fc7fe4fac36fc39d&fccid=cee5eb240fcda90f&vjs=3
https://www.indeed.com/rc/clk?jk=fcd8f59113385b5d&fccid=e490ccf806951166&vjs=3
https://www.indeed.com/rc/clk?jk=75cf426272cffb60&fccid=4027cfd917e1ee29&vjs=3
https://www.indeed.com/rc/clk?jk=6979e0f72b393b02&fccid=f89deb5a97c7738a&vjs=3


In [103]:
#https://stackoverflow.com/questions/36864690/iterate-through-a-dataframe-by-index
for i, row in df.iterrows():
    url = row["Url"]
    page = bsoup(requests.get(df.iloc[i]["Url"]).text, "html.parser")
    df.iloc[i]["Position"] = page.find("h3",class_="icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title").string
    df.iloc[i]["Description"] = page.find("div",class_="jobsearch-JobComponent-description icl-u-xs-mt--md").text

In [105]:
df.head(10)

,Position,Description,Url
0,Senior Machine Learning Engineer,What is Pathrise?\nPathrise (YC W18) invests i...,https://www.indeed.com/rc/clk?jk=ae20548c6763d...
1,Machine Learning Engineer,CircleUp harnesses the power of machine learni...,https://www.indeed.com/rc/clk?jk=2f995bb9c1a1d...
2,Data Scientist - Applied Machine Learning Engi...,The world runs on software. Yet delivering cha...,https://www.indeed.com/rc/clk?jk=09e77269e281b...
3,Machine Learning Engineer - Recommendation Ser...,"At Change we are a unique blend of engineers, ...",https://www.indeed.com/rc/clk?jk=1848b64b8db92...
4,Senior Software Engineer - Machine Learning,About Bill.com\nBill.com is the leading busine...,https://www.indeed.com/rc/clk?jk=47f8abbfe2603...
5,Sr. Machine Learning Engineer / Data Scientist...,About Us\n\nTuro is a peer-to-peer car sharing...,https://www.indeed.com/rc/clk?jk=c0930d01caa5d...
6,Senior Machine Learning Engineer,We are looking for an experienced Software Eng...,https://www.indeed.com/rc/clk?jk=fc7fe4fac36fc...
7,Machine Learning Engineer,"At Toyota Research Institute (TRI), we’re work...",https://www.indeed.com/rc/clk?jk=fcd8f59113385...
8,Lead or Principal Machine Learning Engineer,Job Category\nSalesforce.org - Products and Te...,https://www.indeed.com/rc/clk?jk=75cf426272cff...
9,"Full Stack, Software Engineer",The challenge\nIn the mission towards continuo...,https://www.indeed.com/rc/clk?jk=6979e0f72b393...


In [109]:
df.to_csv(indeed_mle_csv, encoding='utf-8', index=False)

In [111]:
temp = bsoup(requests.get(df.iloc[0]["Url"]).text, "html.parser")
print(page.find("div",class_="jobsearch-JobComponent-description icl-u-xs-mt--md").text)

Rakuten Intelligence is looking for a Data Scientist / Machine Learning Engineer to join our Information Extraction team.

The Information Extraction team at Slice is responsible for using machine learning to build the world's largest cross-merchant purchase graph from email. We build engines for email classification and field extraction. We leverage existing open source technologies and established machine learning approaches, but also innovate and research new techniques. And of course, we do all this at a large scale: hundreds of billions of documents.

As a data scientist / machine learning engineer, you will spend a mix of your time researching new solutions for information extraction as well as productionizing these solutions to make sure they work at scale. You will work with world-class data scientists who hold MS/PhD degrees from top global schools such as Stanford University, University of Washington, Carnegie Mellon University, and National Taiwan University, and veterans fr